In [3]:
import os
import random
import pretty_midi
from tqdm import tqdm


def load_midi(file_path):
    return pretty_midi.PrettyMIDI(file_path)

def is_chord(note_events):
    return len(note_events) > 1

def modify_chord(chord_notes):
    # Sort notes by pitch in descending order
    chord_notes.sort(key=lambda note: note.pitch, reverse=True)
    highest_pitch = chord_notes[0].pitch

    # Keep highest pitch the same, change others within range
    for note in chord_notes[1:]:
        note.pitch += random.randint(-24, 24)

    # Ensure highest pitch note remains the same
    chord_notes[0].pitch = highest_pitch

def process_midi(file_path, output_path):
    midi_data = load_midi(file_path)
    for instrument in midi_data.instruments:
        if instrument.is_drum:
            continue
        
        active_notes = []
        for note in instrument.notes:
            # Check if current notes are overlapping (part of a chord)
            if active_notes and note.start < active_notes[-1].end:
                active_notes.append(note)
            else:
                # Process the current chord
                if is_chord(active_notes):
                    modify_chord(active_notes)
                active_notes = [note]  # Start a new chord
        
        # Modify any remaining chord after loop
        if is_chord(active_notes):
            modify_chord(active_notes)

    # Write out the modified MIDI file
    midi_data.write(output_path)
    print(f"Modified MIDI saved to: {output_path}")

def run_harmonize_random_notes(midi_file_paths):
    for midi_file_path in tqdm(midi_file_paths):
        output_folder = os.path.dirname(midi_file_path)
        output_path = os.path.join(output_folder, "random_chords.mid")
        process_midi(midi_file_path, output_path)
        print(f"Processed file: {midi_file_path}")
    print("All MIDI files processed")

In [4]:
run_harmonize_random_notes(["/homes/kb658/fusion/input/Monophonic_Johnny Marks - Rudolph The Red-Nosed Reindeer.mid"])

100%|██████████| 1/1 [00:00<00:00,  6.16it/s]

Modified MIDI saved to: /homes/kb658/fusion/input/random_chords.mid
Processed file: /homes/kb658/fusion/input/original_Johnny Marks - Rudolph The Red-Nosed Reindeer.mid
All MIDI files processed


In [2]:
import os
import random
import pretty_midi
from tqdm import tqdm


def load_midi(file_path):
    return pretty_midi.PrettyMIDI(file_path)


def generate_random_chord(melody_note, num_notes=3, pitch_range=12):
    """
    Generate a random chord for a given melody note.
    - `num_notes`: Number of additional notes in the chord.
    - `pitch_range`: Range of semitones to choose for harmonizing notes.
    """
    chord_notes = []
    for _ in range(num_notes):
        offset = random.randint(-pitch_range, pitch_range)
        chord_pitch = melody_note.pitch + offset
        # Create a new note based on the melody note
        harmonized_note = pretty_midi.Note(
            velocity=melody_note.velocity,
            pitch=chord_pitch,
            start=melody_note.start,
            end=melody_note.end,
        )
        chord_notes.append(harmonized_note)
    return chord_notes


def harmonize_melody(file_path, output_path, num_chord_notes=3, pitch_range=12):
    midi_data = load_midi(file_path)
    for instrument in midi_data.instruments:
        if instrument.is_drum:
            continue
        
        new_notes = []
        for note in instrument.notes:
            # Keep the melody note
            new_notes.append(note)
            if random.random() < 0.5:
                # Add harmonizing random chord notes
                random_chord = generate_random_chord(
                    note, num_notes=num_chord_notes, pitch_range=pitch_range
                )
                new_notes.extend(random_chord)
        
        # Replace the instrument's notes with the harmonized ones
        instrument.notes = new_notes

    # Write the harmonized MIDI file
    midi_data.write(output_path)
    print(f"Harmonized MIDI saved to: {output_path}")


def run_harmonize_random_chords(midi_file_paths, num_chord_notes=3, pitch_range=12):
    for midi_file_path in tqdm(midi_file_paths):
        output_folder = os.path.dirname(midi_file_path)
        output_path = os.path.join(output_folder, "harmonized_melody.mid")
        harmonize_melody(midi_file_path, output_path, num_chord_notes, pitch_range)
        print(f"Processed file: {midi_file_path}")
    print("All MIDI files processed.")


# Example usage
midi_files = ["/homes/kb658/fusion/input/monophonic_Dave Brubeck, Iola Brubeck - When I Was Young.mid"]
run_harmonize_random_chords(midi_files, num_chord_notes=3, pitch_range=24)


100%|██████████| 1/1 [00:00<00:00, 17.15it/s]

Harmonized MIDI saved to: /homes/kb658/fusion/input/harmonized_melody.mid
Processed file: /homes/kb658/fusion/input/monophonic_Dave Brubeck, Iola Brubeck - When I Was Young.mid
All MIDI files processed.


In [22]:
import os
import random
import pretty_midi
from tqdm import tqdm
os.sys.path.append('/homes/kb658/fusion')
from utils.utils import xml_to_monophonic_midi

musicxml_file = "/homes/kb658/fusion/input/Peter Drischel - Slow Motion (Transcribed Blues Sax).mxl"
midi_output_file = "/homes/kb658/fusion/input/monophonic_Peter Drischel - Slow Motion (Transcribed Blues Sax).mid"

xml_to_monophonic_midi(musicxml_file, midi_output_file)

Monophonic melody saved as /homes/kb658/fusion/input/monophonic_Peter Drischel - Slow Motion (Transcribed Blues Sax).mid


In [16]:
import pretty_midi

def modify_midi_track(input_file, output_file, start_time, end_time, new_track_index):
    """
    Modifies a MIDI file, changing the track of all notes between start_time and end_time
    to the specified new_track_index while preserving the instrument.

    Args:
        input_file (str): Path to the input MIDI file.
        output_file (str): Path to save the modified MIDI file.
        start_time (float): Start time in seconds for the range to modify.
        end_time (float): End time in seconds for the range to modify.
        new_track_index (int): The new track index for notes within the specified time range.
    """
    # Load the MIDI file
    midi_file = pretty_midi.PrettyMIDI(input_file)
    
    # Create a new PrettyMIDI object for the output
    output_midi = pretty_midi.PrettyMIDI()
    
    # Iterate through all instruments in the MIDI file
    for instrument in midi_file.instruments:
        # Create a new instrument for output (with the same program and is_drum property)
        modified_instrument = pretty_midi.Instrument(program=new_track_index, is_drum=instrument.is_drum)
        original_instrument = pretty_midi.Instrument(program=instrument.program, is_drum=instrument.is_drum)
        
        # Separate notes into modified and unmodified based on the time range
        for note in instrument.notes:
            if start_time <= note.start <= end_time:
                # Add note to the modified instrument (simulating the new track)
                modified_instrument.notes.append(note)
            else:
                # Add note to the original instrument
                original_instrument.notes.append(note)
        
        # Add both instruments to the output MIDI
        output_midi.instruments.append(original_instrument)
        output_midi.instruments.append(modified_instrument)
    
    # Save the modified MIDI file
    output_midi.write(output_file)


# Modify all notes between 20 and 40 seconds to track 1, saving to a new file
# Usage
modify_midi_track("/homes/kb658/fusion/input/generated_original_10.mid", "/homes/kb658/fusion/input/generated_original_10_separated.mid", start_time=20, end_time=40, new_track_index=1)


In [15]:
import pretty_midi

def print_unique_programs(midi_file):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_file)

    # Create a set to store unique programs
    unique_programs = set()
    unique_instruments = set()

    # Iterate over all instruments
    for instrument in midi_data.instruments:
        # Iterate over all notes in the instrument
        for note in instrument.notes:
            # Add the program to the set
            unique_programs.add(instrument.program)
            unique_instruments.add(instrument.name)

    # Print the unique programs
    for program in unique_programs:
        print(f"Program: {program}")

    # Print the unique instruments
    for instrument in unique_instruments:
        print(f"Instrument: {instrument}")

# Usage
midi_file = "/homes/kb658/fusion/input/amt_inpainted_separated.mid"
print_unique_programs(midi_file)


Program: 0
Program: 1
Instrument: 


In [17]:
import pretty_midi

def modify_midi(input_file, output_file, start_time, end_time):
    """
    Modifies a MIDI file, changing the track of all notes between start_time and end_time
    to the specified new_track_index while preserving the instrument.

    Args:
        input_file (str): Path to the input MIDI file.
        output_file (str): Path to save the modified MIDI file.
        start_time (float): Start time in seconds for the range to modify.
        end_time (float): End time in seconds for the range to modify.
        new_track_index (int): The new track index for notes within the specified time range.
    """
    # Load the MIDI file
    midi_file = pretty_midi.PrettyMIDI(input_file)
    
    # Create a new PrettyMIDI object for the output
    output_midi = pretty_midi.PrettyMIDI()
    
    # Iterate through all instruments in the MIDI file
    for instrument in midi_file.instruments:
        # Create the original instrument for output (with the same program and is_drum property)
        original_instrument = pretty_midi.Instrument(program=instrument.program, is_drum=instrument.is_drum)
        
        # Separate notes into modified and unmodified based on the time range
        for note in instrument.notes:
            if start_time <= note.start <= end_time:
                # Do not add note to the original instrument
                continue
            else:
                # Add note to the original instrument
                original_instrument.notes.append(note)
        
        # Add both instruments to the output MIDI
        output_midi.instruments.append(original_instrument)
    
    # Save the modified MIDI file
    output_midi.write(output_file)


# Modify all notes between 20 and 40 seconds to track 1, saving to a new file
# Usage
modify_midi("/homes/kb658/fusion/input/original_original_10.mid", "/homes/kb658/fusion/input/original_original_10_separated.mid", start_time=20, end_time=40)
